This notebook is based on Andrew Ng's machine learning course on coursera

For this experiment, we will identify hand written digits based on multi class logistic regression. The idea is very simple: for each digit, we will train a logistic regression model to predict whether the image is the digit or not.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat #tool for loading the data

# Data loading
The data type is mat, so we need the scipy tool to load it.

In [3]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

Each row in X represents the pixels in a 20 x 20 image. And y represents the digit label for each row.

# Sigmoid function
The sigmoid function can be written as:
$$g(z)=\frac{1}{1+{{e}^{-z}}}$$

Given the input matrix X and coefficient $\theta$, the sigmoid funtion for logistic regression is:
$${{h}_{\theta}}(x)=\frac{1}{1+{{e}^{-{{\theta}^{T}}X}}}$$

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Cost function
$$J(\theta)=\frac{1}{m}\sum_{i=1}^{m}{[-{{y}^{(i)}}\log({{h}_{\theta}}({{x}^{(i)}}))-(1-{{y}^{(i)}})\log(1-{{h}_{\theta}}({{x}^{(i)}}))]}+\frac{\lambda}{2m}\sum_{j=1}^{n}{\theta _{j}^{2}}$$

In [6]:
def cost(theta, X, y, learningRate,lambda_):
    #theta: 1*401 vector 
    #lambda_: regularization hyperparameter
    
    # STEP1：transform the X, y, and theta to np.matrix
    # your code here  (appro ~ 3 lines)
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP2：calculate the binary cross entropy loss
    # your code here  (appro ~ 2 lines)
    h = sigmoid(X@theta.T)
    cross_cost = (-y.T@np.log(h) - (1-y).T@np.log(1-h))/len(y)
    
    # STEP3：calculate the regularization loss
    # your code here  (appro ~ 1 lines)
    reg = lambda_*np.sum(np.power(theta.T,2))/(2*len(y))
   
    # STEP4：sum the binary cross entropy loss and regularization loss
    # your code here  (appro ~ 1 lines)
    whole_cost = np.sum(cross_cost)+reg
    
    return whole_cost

# Gradient Descent

Repeat until convergence{
$$\theta_{0}:= \theta_{0} - \alpha\frac{1}{m}\sum_{i=1}^{m}[h_{\theta}(x^{i}-y^{i})]x_{0}^{i}$$
$$\theta_{j}:= \theta_{j} - \alpha\frac{1}{m}\sum_{i=1}^{m}[h_{\theta}(x^{i}-y^{i})]x_{0}^{i} + \frac{\lambda}{m}\theta_{j}$$
}



In [7]:
def gradient(theta, X, y, learningRate, lambda_):
    #theta: 1*401 vector 
    #lambda_: regularization hyperparameter

    
    # STEP1：transform the X, y, and theta to np.matrix
    # your code here  (appro ~ 3 lines)
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP2：flatten the theta
    # your code here  (appro ~ 1 lines)
    parameters = theta.ravel()
    
    # STEP3：calculate the error
    # your code here  (appro ~ 1 lines)    
    error = sigmoid(X@theta.T) - y #N*1
    
    # STEP4：calculate the gradient
    # your code here  (appro ~ 1 lines)
    grad = learningRate*error.T@X/len(y) + lambda_*parameters/len(y) 
    
    # STEP5：no regularization when j = 0
    # your code here  (appro ~ 1 lines)
    grad[0, 0] = learningRate*error.T@X[:,0]/len(y)
    
    return np.array(grad).ravel()

# Classifier

For this task, we have 10 possible classes so we need multi-class classification strategy as logistic regression can only deal with two classes. In this experiment, we are going to build 10 different classifiers for 10 different classes and each classifier will decide whether this datapoint belongs to one class or not. 

In the next cell, we will wrap up the training process in the 'one_vs_all' function and return the 10 thetas for 10 classifiers. Each theta will have 401 elements because we have 401 parameters (including $x_{0}$).

In [8]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate, lambda_= 1):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k * (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1) 
        y_i = np.array([1 if label == i else 0 for label in y]) #change y_i to 1 or 0 for class i
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate,lambda_), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

# Training

In [9]:
#this cell is to show the shape of the parameters.
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [10]:
np.unique(data['y'])#the label of 10 classes

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [13]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1, 1)
all_theta

array([[-2.27399874e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.20284906e-03,  2.20150236e-08,  0.00000000e+00],
       [-3.14172491e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.26184428e-03, -4.87250372e-04,  0.00000000e+00],
       [-4.79619818e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86767445e-05, -2.46938190e-07,  0.00000000e+00],
       ...,
       [-7.92220923e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.59664793e-05,  7.35429645e-06,  0.00000000e+00],
       [-4.57137645e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33530143e-03,  9.98857549e-05,  0.00000000e+00],
       [-4.94998167e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.11004232e-04,  9.92319880e-06,  0.00000000e+00]])

# Prediction

In [17]:
def predict_all(X, all_theta):
    # TODO：prediction for test data
    
    # STEP1：
    rows = X.shape[0] #number of datapoints
    params = X.shape[1] #number of parameters
    num_labels = all_theta.shape[0] #number of labels
    
    # STEP2：Insert a column of all ones to X
    # your code here  (appro ~ 1 lines)
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # STEP3：Transform X and all_theta to numpy matrix
    # your code here  (appro ~ 2 lines)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # STEP4：Calculate the probability of X in all 10 classes
    # your code here  (appro ~ 1 lines)
    h = sigmoid(X@all_theta.T)
    
    # STEP5：Find the maximum probability for each data point
    # your code here  (appro ~ 1 lines)
    h_argmax = np.argmax(h, axis=1)
    
    # STEP6：Add one to get the real label (python is 0-indexed)
    h_argmax = h_argmax + 1
    
    return h_argmax

In [18]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.44%


If you are right, your accuracy should be larger than 90%